In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as tf_keras

In [ ]:
%%bash

cp drive/MyDrive/Colab\ Notebooks/data-files/dogs-vs-cats.zip sample_data/
unzip sample_data/dogs-vs-cats.zip -d sample_data
unzip sample_data/dogs-vs-cats/train.zip -d sample_data/dogs-vs-cats
unzip sample_data/dogs-vs-cats/test1.zip -d sample_data/dogs-vs-cats
mv sample_data/dogs-vs-cats/test1 sample_data/dogs-vs-cats/test
rm -rf sample_data/__MACOSX

In [6]:
import os
import shutil

src_base = 'sample_data/dogs-vs-cats'
dest_base = 'sample_data/dogs-vs-cats2'

if os.path.exists(dest_base): # 경로 존재여부 확인
  shutil.rmtree(dest_base) # train 경로 및 하위 경로 삭제
os.mkdir(dest_base) # 디렉터리 만들기

# train 폴더의 0 ~ 1000 : train, 1000 ~ 1500 : validation, 1500 ~ 2000 : test 세트로 구성
for start, stop, path in zip([0, 1000, 1500], [1000, 1500, 2000], ['train', 'validation', 'test']):
  os.mkdir(os.path.join(dest_base, path))
  for sub_path in ['cat', 'dog']:
    os.mkdir(os.path.join(dest_base, path, sub_path))
    for idx in np.arange(start, stop):
      fname = f'{sub_path}.{idx}.jpg'
      shutil.copy(os.path.join(src_base, 'train', fname), os.path.join(dest_base, path, sub_path, fname)) # 파일 복사

In [15]:
# 입력 자동화 도구 만들기 : 파일을 읽어서 모델에 입력 가능한 형식으로 변환하는 도구

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(rescale=1/255) # 특정 디렉터리의 파일을 읽어서 모델의 입력데이터로 변환하는 도구
validation_data_generator = ImageDataGenerator(rescale=1/255)
test_data_generator = ImageDataGenerator(rescale=1/255, rotation_range=360)

In [16]:
# generator를 사용해서 파일 데이터 읽기
datasets = []
for path, generator in zip(['train', 'validation', 'test'],
                           [train_data_generator, validation_data_generator, test_data_generator]):
  dataset = generator.flow_from_directory(directory=f"sample_data/dogs-vs-cats2/{path}",
                                          target_size=(256, 256),
                                          batch_size=32,
                                          class_mode="binary")
  datasets.append(dataset)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [9]:
# 이미 학습 완료된 비슷한 모델 활용 ( 전이학습 )

# 1. 다른 모델의 합성곱 층이 출력한 데이터를 현재 모델의 입력으로 사용
# 2. 다른 모델과 현재 모델 결합 1 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층은 학습하지 않음 )
# 3. 다른 모델과 현재 모델 결합 2 (다른 모델의 합성곱 층에 층을 추가해서 새 모델 구성 - 다른 모델의 합성곱 층의 일부는 학습 )

In [22]:
# 사전 학습 모델 준비

base_model = tf_keras.applications.vgg16.VGG16(include_top=False,
                                               weights='imagenet',
                                               input_shape=(256, 256, 3))

base_model.trainable = True

In [23]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [ ]:
[ layer.trainable for layer in base_model.layers ]

In [25]:
for layer in base_model.layers[:-4]:
  layer.trainable = False

In [ ]:
[ layer.trainable for layer in base_model.layers ]

In [32]:
input = tf_keras.layers.Input(shape=(256, 256, 3))
x = base_model(input)
x = tf_keras.layers.Flatten()(x)
x = tf_keras.layers.Dense(units=512, activation="relu")(x)
x = tf_keras.layers.Dropout(rate=0.5)(x) # 과대적합 완화
output = tf_keras.layers.Dense(units=1, activation="sigmoid")(x)

model = tf_keras.models.Model(input, output)

In [33]:
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=['accuracy'])

In [ ]:
histroy = model.fit(datasets[0], epochos=20, validation_data=datasets[1])

In [31]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 dense_4 (Dense)             (None, 8, 8, 512)         262656    
                                                                 
 dense_5 (Dense)             (None, 8, 8, 1)           513       
                                                                 
Total params: 14977857 (57.14 MB)
Trainable params: 7342593 (28.01 MB)
Non-trainable params: 7635264 (29.13 MB)
_________________________________________________________________
